### Approximate the peak 2D function using pytorch and polynomial features

This reproduce the example in the documentation with pytorch.
We use a sklearn pipeline with polynomial features and a pytorch model through skorch.

It needs a bit of work to make it work:

Polynomial features don't work out of the box in skorch, we need to convert the output to float32. So we need a modified one.

We need to register the skorch object and the modified polynomial feature to gurobi_ml.

Extra required packages:
- matplotlib
- skorch


In [ ]:
import gurobipy as gp
import numpy as np
import torch
from skorch import NeuralNetRegressor
from gurobipy import GRB
from matplotlib import cm
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

from gurobi_ml import add_predictor_constr, register_predictor_constr

from gurobi_ml.sklearn import add_polynomial_features_constr
from gurobi_ml.torch import add_sequential_constr

In [ ]:
def peak2d(xx, yy):
    return (
        3 * (1 - xx) ** 2.0 * np.exp(-(xx**2) - (yy + 1) ** 2)
        - 10 * (xx / 5 - xx**4 - yy**5) * np.exp(-(xx**2) - yy**2)
        - 1 / 3 * np.exp(-((xx + 1) ** 2) - yy**2)
    )

In [ ]:
x = torch.arange(-2, 2, 0.01)
y = torch.arange(-2, 2, 0.01)
x1, x2 = torch.meshgrid(x, y, indexing="ij")
z = peak2d(x1, x2)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(x1, x2, z, cmap=cm.coolwarm, linewidth=0.01, antialiased=False)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
X = torch.cat([x1.ravel().reshape(-1, 1), x2.ravel().reshape(-1, 1)], axis=1)
y = z.ravel().reshape(-1, 1)

In [ ]:
# Somehow polynomial features don't work out of the box because
# the output needs to be converted to float32
# Create a small class to apply polynomial features and convert

from sklearn.base import BaseEstimator, TransformerMixin


class MyPolynomialFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.poly_feat = PolynomialFeatures()

    def fit(self, X, y=None):
        self.poly_feat.fit(X, y)
        self.n_features_in_ = self.poly_feat.n_features_in_
        self.n_output_features_ = self.poly_feat.n_output_features_
        return self

    def transform(self, x):
        return self.poly_feat.transform(x).astype(np.float32)

In [ ]:
hs = 16
nn_regression = NeuralNetRegressor(
    torch.nn.Sequential(
        torch.nn.Linear(6, hs),
        torch.nn.ReLU(),
        torch.nn.Linear(hs, hs),
        torch.nn.ReLU(),
        torch.nn.Linear(hs, 1),
    ),
    max_epochs=20,
    lr=0.1,
    iterator_train__shuffle=True,
)
poly_feat = MyPolynomialFeatures()
pipeline = make_pipeline(poly_feat, nn_regression)

In [ ]:
pipeline.fit(X, y)

In [ ]:
X_test = torch.rand((100, 2)) * 2 - 1

In [ ]:
metrics.r2_score(peak2d(X_test[:, 0], X_test[:, 1]), pipeline.predict(X_test))

In [ ]:
metrics.max_error(peak2d(X_test[:, 0], X_test[:, 1]), pipeline.predict(X_test))

In [ ]:
pipeline.predict(X).min()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(
    x1,
    x2,
    pipeline.predict(X).reshape(x1.shape),
    cmap=cm.coolwarm,
    linewidth=0.01,
    antialiased=False,
)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

## Register our objects to gurobi_ml package

Before building the model, we first need to register the two objects
to gurobi machine learning so that add_predictor_constr work.

In [ ]:
# First register the skorch object. We need to:
# - Add a function with appropriate signature this function just calls
#   the function to add a pytorch model on the pytorch model
# - Register that function by associating it to the NeuralNetRegressor class
def add_skorch_constr(gp_model, skorch_model, input_vars, output_vars=None, **kwargs):
    return add_sequential_constr(
        gp_model, skorch_model.module, input_vars, output_vars, **kwargs
    )


register_predictor_constr(NeuralNetRegressor, add_skorch_constr)

In [ ]:
# Now do the same for the polynomial features
def add_my_polynomial_features_constr(gp_model, poly_feat, input_vars, **kwargs):
    return add_polynomial_features_constr(
        gp_model, poly_feat.poly_feat, input_vars, **kwargs
    )


register_predictor_constr(MyPolynomialFeatures, add_my_polynomial_features_constr)

### Do the optimization model

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addMVar((1, 2), lb=-2, ub=2, name="x")
y = m.addMVar(1, lb=-GRB.INFINITY, name="y")

m.setObjective(y.sum(), gp.GRB.MINIMIZE)

# Add network trained by pytorch to Gurobi model to predict y from x
nn2gurobi = add_predictor_constr(m, pipeline, x, y)

nn2gurobi.print_stats()

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 10
m.Params.MIPGap = 0.1
m.Params.NonConvex = 2

In [ ]:
m.optimize()

### Look at the solution

In [ ]:
x.X

In [ ]:
peak2d(x.X[0, 0], x.X[0, 1])

In [ ]:
y.X

Copyright © 2023 Gurobi Optimization, LLC